In [382]:
from datasets import load_dataset, DatasetDict, load_from_disk, Dataset
import json
from pathlib import Path
from pprint import pprint
import pandas as pd
from itertools import groupby
from funcy import flatten
from tqdm.autonotebook import tqdm

In [434]:
import warnings, wikipedia
warnings.simplefilter("ignore")

from functools import cache

@cache
def search_new(subject: str, auto_suggest=False, redirect=False):
    try:
      result = wikipedia.summary(subject, auto_suggest=auto_suggest, redirect=redirect).strip(), "success"
    except wikipedia.DisambiguationError as d:
      result = f"\"{subject}\" may refer to one of {d.args[1]}. Please retry the search with one of the subjects using Search[<subject>].", "disambg"
    except wikipedia.PageError as e:
      result = f"{e} Please retry the search using Search[<subject>].", "pageerror"
    except wikipedia.WikipediaException as e:
      print(e, type(e))
      result = str(e), f"other:{type(e)}"
    except Exception as e:
      print(e, type(e))
      result = str(e), f"other:{type(e)}"
    return result

In [407]:
wikipedia.page("Psych", auto_suggest=False, redirect=True).summary

'Psych is an American detective comedy-drama television series created by Steve Franks for USA Network. The series stars James Roday as Shawn Spencer, a young crime consultant for the Santa Barbara Police Department whose "heightened observational skills" and impressive eidetic memory allow him to convince people that he solves cases with psychic abilities. The program also stars Dulé Hill as Shawn\'s intelligent best friend and reluctant partner Burton "Gus" Guster, as well as Corbin Bernsen as Shawn\'s father Henry, a former detective with the Santa Barbara Police Department.\nPsych premiered on July 7, 2006, following the fifth-season premiere of Monk, and continued to be paired with the series until Monk\'s conclusion on December 4, 2009. During the second season, an animated segment titled "The Big Adventures of Little Shawn and Gus" was added to the series. Psych was the highest-rated US basic cable television premiere of 2006. USA Network renewed the series for an eighth season 

In [404]:
wikipedia.summary("Psych", auto_suggest=False, redirect=False)

'Psych is an American detective comedy-drama television series created by Steve Franks for USA Network. The series stars James Roday as Shawn Spencer, a young crime consultant for the Santa Barbara Police Department whose "heightened observational skills" and impressive eidetic memory allow him to convince people that he solves cases with psychic abilities. The program also stars Dulé Hill as Shawn\'s intelligent best friend and reluctant partner Burton "Gus" Guster, as well as Corbin Bernsen as Shawn\'s father Henry, a former detective with the Santa Barbara Police Department.\nPsych premiered on July 7, 2006, following the fifth-season premiere of Monk, and continued to be paired with the series until Monk\'s conclusion on December 4, 2009. During the second season, an animated segment titled "The Big Adventures of Little Shawn and Gus" was added to the series. Psych was the highest-rated US basic cable television premiere of 2006. USA Network renewed the series for an eighth season 

In [308]:
search("Kentucky inmate executions")

('Page id "kentucky inmates execution" does not match any pages. Try another id! Please retry the search using Search[<subject>].',
 'pageerror')

In [255]:
search('what "Gray Matters" means')

("Audience analysis is a task that is often performed by technical writers in a project's early stages. It consists of assessing the audience to make sure the information provided to them is at the appropriate level.  The audience is often referred to as the end-user, and all communications need to be targeted towards the defined audience.  Defining an audience requires the consideration of many factors, such as age, culture and knowledge of the subject.  After considering all the known factors, a profile of the intended audience can be created, allowing writers to write in a manner that is understood by the intended audience.",
 'success')

In [133]:
fever = load_dataset("fever/fever", "v1.0")
js = json.loads(Path("task.json").read_text())

In [11]:
len(fever["train"]["id"])

311431

In [12]:
len(set(fever["train"]["id"]))

145449

In [135]:
def mapper(row):
    if row["label"] == "SUPPORTS":
        bool_label = True
    elif row["label"] == "REFUTES":
        bool_label = False
    else:
        print(row["label"])
        raise ValueError("WOW! That's unexpected.")

    wikipedia_title = row["evidence_wiki_url"].replace("_", " ").replace("-LRB-", "(").replace("-RRB-", ")")
    in_bigbench = any(row["claim"] in y["input"] for y in js["examples"])
    return {"bool_label": bool_label, "wikipedia_title": wikipedia_title, "in_bigbench": in_bigbench}
ds = fever.filter(lambda x: x["label"] in ["SUPPORTS", "REFUTES"]).map(mapper, num_proc=32)

Map (num_proc=32):   0%|          | 0/28625 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/14475 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/14150 [00:00<?, ? examples/s]

In [136]:
len(set(ds["train"]["wikipedia_title"]))

12549

In [137]:
def trajectorize(row):
    claim = row["claim"].strip()
    task = f"On June 2017, the following claim was made: {claim}\nQ: Was this claim true or false?"
    answer = row["bool_label"]
    wiki, worked = search(row["wikipedia_title"])
    wiki = wiki.strip()

    trajectory = [
        {"task": task},
        {"thought": f"I need to search {row['wikipedia_title']}."},
        {"action": f"Search[{row['wikipedia_title']}]"},
        {"observation": f"[Document]\n{wiki}\n[End]"},
        {"thought": f"The claim is {answer}."},
        {"action": f"Finish[{answer}]"},
    ]

    traj_keys = [next(iter(t.keys())) for t in trajectory]
    traj_values = [next(iter(t.values())) for t in trajectory]

    rewoo_trajectory = [
        {"task": task},
        {"thought": f"Search for more information about {row['wikipedia_title']}."},
        {"action": f"Search[{row['wikipedia_title']}]"},
        {"observation": f"[Document]\n{wiki}\n[End]"},
    ]

    rewoo_traj_keys = [next(iter(t.keys())) for t in rewoo_trajectory]
    rewoo_traj_values = [next(iter(t.values())) for t in rewoo_trajectory]

    return {
        "traj_keys": traj_keys,
        "traj_values": traj_values,
        "rewoo_traj_keys": rewoo_traj_keys,
        "rewoo_traj_values": rewoo_traj_values,
        "wiki_worked": worked,
        "wiki": wiki,
    }

ds["train_react"] = (
    ds["train"]
    .shuffle()
    .select(range(5000))
    .filter(lambda x: x["in_bigbench"] == False)
    .map(trajectorize, num_proc=32)
)
ds["train_react_wiki_positive"] = ds["train_react"].filter(
    lambda x: x["wiki_worked"] == True
)

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/4999 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4999 [00:00<?, ? examples/s]

## Redo based off original FEVER

In [458]:
fever = load_from_disk("var/fever_reprocessed")
fever

DatasetDict({
    train: Dataset({
        features: ['label', 'claim', 'unique_evidence', 'evidence_sentences', 'evidence_sentence_count', 'id'],
        num_rows: 6720
    })
    test: Dataset({
        features: ['label', 'claim', 'unique_evidence', 'evidence_sentences', 'evidence_sentence_count', 'id'],
        num_rows: 6612
    })
})

In [459]:
def clean_fever(text) -> str:
    mapping = {"_": " ",
               "-LRB- ": "(",
               " -RRB-": ")",
               "-LSB- ": "[",
               " -RSB-": "]",
               "-LRB-": "(",
               "-RRB-": ")",
               "-LSB-": "[",
               "-RSB-": "]",
               "-COLON-": ":"
               }
    
    for k,v in mapping.items():
        text = text.replace(k, v)

    return text.strip()

In [460]:
def trajectorize(row):
    evidence_sentences = row["evidence_sentences"]

    claim = row["claim"].strip()
    task = f"On June 2017, the following claim was made: {claim}\nQ: Was this claim true or false?"
    answer = str(row["label"]).lower()

    article_sentence_group = { k:list(v) for k,v in groupby(evidence_sentences, lambda x: x[0]) }

    articles = {}
    statuses = []
    wiki_worked = True
    for article in article_sentence_group:
        cleaned_article = clean_fever(article)
        wiki, worked = search(article)
        wiki = wiki.strip()
        if worked != "success":
            wiki_worked = False
            # if article in wiki_pages_df.index:
            #     # lines = wiki_pages_df.loc[article]["lines_split"]
            #     # lines = [ clean_fever(l.split('\t')[0]) for l in lines ]
            #     # wiki = "\n".join(lines).strip().replace("\n", " ")
            #     wiki = wiki_pages_df.loc[article]["text"]

            #     worked = "fallback"
            # else:
            #     worked = "fallback_fail"

        articles[cleaned_article] = wiki
        statuses.append(worked)
    all_wiki_success = all(x in ["success","fallback"] for x in statuses)

    trajectory = [
        {"task": task}]
    
    for article,evidences in article_sentence_group.items():
        cleaned_article = clean_fever(article)
        trajectory.extend([
        {"thought": f"I need to search {cleaned_article}."},
        {"action": f"Search[{cleaned_article}]"},
        {"observation": f"[Document]\n{articles[cleaned_article]}\n[End]"}])

        for title,line,sent in evidences:
            trajectory.append(
        {"observation": clean_fever(sent.split('\t')[0])})
    
    trajectory.extend([
        {"thought": f"The claim is {answer}."},
        {"action": f"Finish[{answer}]"},
    ])

    traj_keys = [next(iter(t.keys())) for t in trajectory]
    traj_values = [next(iter(t.values())) for t in trajectory]

    rewoo_trajectory = [
        {"task": task}]
    
    for article,evidences in article_sentence_group.items():
        cleaned_article = clean_fever(article)
        rewoo_trajectory.extend([
        {"thought": f"Search for more information about {cleaned_article}."},
        {"action": f"Search[{cleaned_article}]"},
        {"observation": f"[Document]\n{articles[cleaned_article]}\n[End]"}])

        for title,line,sent in evidences:
            rewoo_trajectory.append(
        {"observation": clean_fever(sent.split('\t')[0])})

    rewoo_traj_keys = [next(iter(t.keys())) for t in rewoo_trajectory]
    rewoo_traj_values = [next(iter(t.values())) for t in rewoo_trajectory]

    return {
        "traj_keys": traj_keys,
        "traj_values": traj_values,
        "rewoo_traj_keys": rewoo_traj_keys,
        "rewoo_traj_values": rewoo_traj_values,
        "all_wiki_success": all_wiki_success,
        "wiki_worked": wiki_worked,
        "articles": list(articles.values()),
        "statuses": statuses
    }

In [461]:
def sentencify(row):
    evidence_sentences = row["evidence_sentences"]

    article_sentence_group = { clean_fever(k):list(v) for k,v in groupby(evidence_sentences, lambda x: x[0]) }

    sentences = []
    for article,evidences in article_sentence_group.items():
        for title,line,sent in evidences:
            sentences.append(clean_fever(sent.split('\t')[0]))

    return {
        "cot": " ".join(sentences).strip().replace("\n", " ")
    }

In [462]:
fever["train"] = (
    fever["train"]
    # .select(range(500))
    .map(trajectorize, num_proc=16)
    .map(sentencify, num_proc=16)
)

Map (num_proc=16):   0%|          | 0/6720 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/6720 [00:00<?, ? examples/s]

: 

In [457]:
fever.save_to_disk("var/fever_augmented_nowikipages")

Saving the dataset (0/1 shards):   0%|          | 0/6720 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6612 [00:00<?, ? examples/s]

In [341]:
wiki_reworked = fever["train"].filter(lambda x: x["wiki_worked"] == False).map(trajectorize, num_proc=8)

Parameter 'function'=<function trajectorize at 0x7c7742f20> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map (num_proc=8):   0%|          | 0/2116 [00:00<?, ? examples/s]

In [449]:
failed = fever["train"].filter(lambda x: x["wiki_worked"] == False).to_pandas()

Filter:   0%|          | 0/6720 [00:00<?, ? examples/s]

In [450]:
failed

,label,claim,unique_evidence,evidence_sentences,evidence_sentence_count,id,cot,traj_keys,traj_values,rewoo_traj_keys,rewoo_traj_values,all_wiki_success,wiki_worked,articles,statuses
0,True,Trollhunters was produced by an animation comp...,"[[Trollhunters, 0]]","[[Trollhunters, 0, Trollhunters is an American...",1,60685,Trollhunters is an American computer-animated ...,"[task, thought, action, observation, observati...","[On June 2017, the following claim was made: T...",[task],"[On June 2017, the following claim was made: T...",False,False,"[Page id ""troll hunters"" does not match any pa...",[pageerror]
1,True,DreamWorks Animation produced Trollhunters.,"[[Trollhunters, 0]]","[[Trollhunters, 0, Trollhunters is an American...",1,46475,Trollhunters is an American computer-animated ...,"[task, thought, action, observation, observati...","[On June 2017, the following claim was made: D...",[task],"[On June 2017, the following claim was made: D...",False,False,"[Page id ""troll hunters"" does not match any pa...",[pageerror]
2,True,Trollhunters is computer-animated.,"[[Trollhunters, 0]]","[[Trollhunters, 0, Trollhunters is an American...",1,53330,Trollhunters is an American computer-animated ...,"[task, thought, action, observation, observati...","[On June 2017, the following claim was made: T...",[task],"[On June 2017, the following claim was made: T...",False,False,"[Page id ""troll hunters"" does not match any pa...",[pageerror]
3,True,Robert Palmer (writer) has written for the New...,"[[Robert_Palmer_-LRB-writer-RRB-, 1]]","[[Robert_Palmer_-LRB-writer-RRB-, 1, He is bes...",1,3583,"He is best known for his books , including Dee...","[task, thought, action, observation, observati...","[On June 2017, the following claim was made: R...",[task],"[On June 2017, the following claim was made: R...",False,False,"[""Robert Palmer (writer)"" may refer to one of ...",[disambg]
4,True,Robert Palmer (writer) wrote Deep Blues.,"[[Robert_Palmer_-LRB-writer-RRB-, 1]]","[[Robert_Palmer_-LRB-writer-RRB-, 1, He is bes...",1,55999,"He is best known for his books , including Dee...","[task, thought, action, observation, observati...","[On June 2017, the following claim was made: R...",[task],"[On June 2017, the following claim was made: R...",False,False,"[""Robert Palmer (writer)"" may refer to one of ...",[disambg]
5,True,Trollhunters is a television series.,"[[Trollhunters, 0]]","[[Trollhunters, 0, Trollhunters is an American...",1,18523,Trollhunters is an American computer-animated ...,"[task, thought, action, observation, observati...","[On June 2017, the following claim was made: T...",[task],"[On June 2017, the following claim was made: T...",False,False,"[Page id ""troll hunters"" does not match any pa...",[pageerror]
6,False,Robert Palmer (writer) isn't a journalist.,"[[Robert_Palmer_-LRB-writer-RRB-, 1]]","[[Robert_Palmer_-LRB-writer-RRB-, 1, He is bes...",1,130181,"He is best known for his books , including Dee...","[task, thought, action, observation, observati...","[On June 2017, the following claim was made: R...",[task],"[On June 2017, the following claim was made: R...",False,False,"[""Robert Palmer (writer)"" may refer to one of ...",[disambg]
7,False,Trollhunters was only produced by Donald Trump.,"[[Trollhunters, 0]]","[[Trollhunters, 0, Trollhunters is an American...",1,28716,Trollhunters is an American computer-animated ...,"[task, thought, action, observation, observati...","[On June 2017, the following claim was made: T...",[task],"[On June 2017, the following claim was made: T...",False,False,"[Page id ""troll hunters"" does not match any pa...",[pageerror]
8,False,Robert Palmer (writer) has yet to write for a ...,"[[Robert_Palmer_-LRB-writer-RRB-, 1]]","[[Robert_Palmer_-LRB-writer-RRB-, 1, He is bes...",1,58735,"He is best known for his books , including Dee...","[task, thought, action, observation, observati...","[On June 2017, the following claim was made: R...",[task],"[On June 2017, the following claim was made: R...",F

In [373]:
len(list(flatten([ [y[0] for y in x] for x in fever["train"]["unique_evidence"] ])))

7608

In [375]:
articles = list(set(flatten([ [y[0] for y in x] for x in fever["train"]["unique_evidence"] ])))

In [383]:
article_ds = Dataset.from_dict({"article": articles})
article_ds

Dataset({
    features: ['article'],
    num_rows: 1043
})

In [437]:
def searcher(row):
    cleaned = clean_fever(row["article"])
    if row["msg"] != "success":
        wiki, msg = search_new(cleaned, auto_suggest=True, redirect=True)
    else:
        wiki = row["wiki"]
        msg = row["msg"]
    return {"wiki": wiki, "msg": msg, "cleaned": cleaned}
article_ds = article_ds_noauto.map(searcher, num_proc=4)
article_df = article_ds.to_pandas()
article_df[article_df.msg != "success"]

Map (num_proc=4):   0%|          | 0/1043 [00:00<?, ? examples/s]

,article,wiki,msg,cleaned
204,Trollhunters,"Page id ""troll hunters"" does not match any pag...",pageerror,Trollhunters
588,The_Kingdom_-LRB-film-RRB-,"""The Kingdom (film)"" may refer to one of ['The...",disambg,The Kingdom (film)
712,Robert_Palmer_-LRB-writer-RRB-,"""Robert Palmer (writer)"" may refer to one of [...",disambg,Robert Palmer (writer)


In [438]:
article_df = article_df.set_index("article")

In [440]:
article_df.loc["Psych"]

ValueError: too many values to unpack (expected 2)

In [441]:
def search(query):
    row = article_df.loc[query]
    return row["wiki"], row["msg"]

In [436]:
for i in article_df[article_df.msg != "success"]["cleaned"]:
    print(i, wikipedia.summary(i))

Underdog (film) Underdog is a 2007 American superhero comedy film based on W. Watts Biggers, Chet Stover, and Joe Harris‘ 1960s animated television series Underdog, which in turn is a spoof on the DC Comics character Superman, created by Jerry Siegel and Joe Shuster. Directed by Frederik Du Chau and written by Joe Piscatella, Adam Rifkin, and Craig A. Williams, the film stars Jim Belushi, Peter Dinklage, John Slattery, and Patrick Warburton with the voice talents of Jason Lee, Amy Adams, and Brad Garrett. Unlike the TV series, the Underdog character is portrayed as a regular dog rather than an anthropomorphic one. Underdog/Shoeshine, voiced by Jason Lee, was played by a lemon beagle named Leo sporting a red sweater and a blue cape. The film grossed $65.3 million worldwide and received generally negative reviews from critics.
James Jones (basketball player) James Andrew Jones (born October 4, 1980) is an American professional basketball executive and former player. He is both the presid

PageError: Page id "troll hunters" does not match any pages. Try another id!

## End redo

In [129]:
badapple = ds["train_react_wiki_positive"].filter(lambda x: "Bronson (film) is a fictionalized crime film based on someone's life" in x["claim"])

Filter:   0%|          | 0/3336 [00:00<?, ? examples/s]

In [130]:
badapple[0]

{'id': 152602,
 'label': 'SUPPORTS',
 'claim': "Bronson (film) is a fictionalized crime film based on someone's life.",
 'evidence_annotation_id': 341237,
 'evidence_id': 325711,
 'evidence_wiki_url': 'Bronson_-LRB-film-RRB-',
 'evidence_sentence_id': 1,
 'bool_label': True,
 'wikipedia_title': 'Bronson (film)',
 'in_bigbench': False,
 'traj_keys': ['task',
  'thought',
  'action',
  'observation',
  'thought',
  'action'],
 'traj_values': ["On June 2017, the following claim was made: Bronson (film) is a fictionalized crime film based on someone's life.\nQ: Was this claim true or false?",
  'I need to search Bronson (film).',
  'Search[Bronson (film)]',
  "[Document]\nBronson is a 2008 British biographical prison drama film directed by Nicolas Winding Refn, based on a script written by Refn and Brock Norman Brock. The film stars Tom Hardy as Michael Peterson, known from 1987 as Charles Bronson. The film follows the life of this prisoner, considered Britain's most violent criminal, who 

In [138]:
sum(map(int, ds["train_react"]["wiki_worked"]))/len(ds["train_react"])

0.6515303060612122

In [139]:
ds["paper_test_bigbench"] = ds["paper_test"].filter(lambda x: x["in_bigbench"] == True)

Filter:   0%|          | 0/14150 [00:00<?, ? examples/s]

In [140]:
ds.save_to_disk("var/fever_agentic")

Saving the dataset (0/1 shards):   0%|          | 0/263822 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/28625 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards): 0 examples [00:00, ? examples/s]

Saving the dataset (0/1 shards): 0 examples [00:00, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14475 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14150 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4999 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3257 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13582 [00:00<?, ? examples/s]

In [98]:
new_ds = DatasetDict()
for k,v in ds_c.items():
    if len(v) > 0:
        new_ds[k] = v
new_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 263822
    })
    labelled_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 28625
    })
    paper_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 14475
    })
    paper_test: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 14150
    })
})

In [99]:
new_ds_bb = DatasetDict()
for k,v in new_ds.items():
    new_ds_bb[f"{k}_bigbench"] = v.filter(), num_proc=32)
    new_ds_bb[f"{k}_not_bigbench"] = v.filter(lambda x: all(x["claim"] not in y["input"] for y in js["examples"]), num_proc=32)
new_ds_bb

DatasetDict({
    train_bigbench: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 10
    })
    train_not_bigbench: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 263812
    })
    labelled_dev_bigbench: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 13582
    })
    labelled_dev_not_bigbench: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 15043
    })
    paper_dev_bigbench: Dataset({
        features: ['id', 'label', 'c

In [78]:
ref_claim = new_ds.filter(lambda x: any(x["claim"] in y["input"] for y in js["examples"]), num_proc=32)

Filter (num_proc=32):   0%|          | 0/263822 [00:00<?, ? examples/s]

Filter (num_proc=32):   0%|          | 0/28625 [00:00<?, ? examples/s]

Filter (num_proc=32):   0%|          | 0/14475 [00:00<?, ? examples/s]

Filter (num_proc=32):   0%|          | 0/14150 [00:00<?, ? examples/s]

In [94]:
new_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 263822
    })
    labelled_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 28625
    })
    paper_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 14475
    })
    paper_test: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 14150
    })
})

In [67]:
ref_claim

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 10
    })
    labelled_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 13582
    })
    paper_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 0
    })
    paper_test: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 13582
    })
})

In [89]:
ex = search("spain")

In [91]:
ex.args

('split',
 ['Split, Croatia',
  'Split Island, Canada',
  'Split Island, Falkland Islands',
  'Split Island, Fiji',
  'Split (1989 film)',
  'Split (2016 American film)',
  'Split (2016 Canadian film)',
  'Split (2016 South Korean film)',
  'Split: A Divided America',
  'The Split (film)',
  'The Manster',
  'Split (poker)',
  'Split (blackjack)',
  'Split (The Groundhogs album)',
  'Split (Lush album)',
  'Split (Patricia Barber album)',
  'The Split CD',
  '"Split" (KMFDM song)',
  '"Split" (Tesla Boy song)',
  '"Split" (Yeat song)',
  'Split/Whole Time',
  'Coping Mechanism',
  'Split (composition)',
  'Split album',
  'Split (TV series)',
  'The Split (TV series)',
  '"Split" (Coupling episode)',
  'Split (novel)',
  'Split (sculpture)',
  'Split Dynasty',
  'Yugoslav destroyer Split',
  'Yugoslav frigate Split',
  'Split (graph theory)',
  'Split (mathematics)',
  'Split Lie algebra',
  'Split (phylogenetics)',
  'split (Unix)',
  'Split (bowling)',
  'Split (gymnastics)',
  'Spli

In [ ]:
def rewoo_trajectory(row):
    claim = row["claim"]
    task = f"On June 2017, the following claim was made: {claim}\nQ: Was this claim true or false?"
    answer = row["bool_label"]
    wiki, worked = search(row["wikipedia_title"])
    trajectory = [
        {"task": task.strip()},
        {"thought": f"Search for more information about {row['wikipedia_title']}."},
        {"action": f"Search[{row['wikipedia_title']}]"},
        {"observation": f"[Document]\n{wiki}\n[End]"},
    ]

    traj_keys = [next(iter(t.keys())) for t in trajectory]
    traj_values = [next(iter(t.values())) for t in trajectory]

    return {
            "rewoo_traj_keys": traj_keys,
            "rewoo_traj_values": traj_values,
        }

ref_claim_react = ref_claim_react.map(rewoo_trajectory).filter(lambda x: x["wiki_worked"])

In [82]:
ref_claim_react

Dataset({
    features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title', 'traj_keys', 'traj_values', 'wiki_worked'],
    num_rows: 4
})

In [93]:
import random


ref_claim_t["test"][random.randint(0,4)]["claim"]

'Fox 2000 Pictures released the film Soul Food.'

In [83]:
ref_claim_t = DatasetDict()
ref_claim_t["train"] = ref_claim_react
ref_claim_t["test"] = new_ds["paper_dev"]
ref_claim_t.save_to_disk("var/ref_claim_fever_react")

Saving the dataset (0/1 shards):   0%|          | 0/4 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14475 [00:00<?, ? examples/s]

In [63]:
ref_claim.save_to_disk("var/ref_claim_fever")

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13582 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards): 0 examples [00:00, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13582 [00:00<?, ? examples/s]

In [46]:
ds_c

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 263822
    })
    labelled_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wikipedia_title'],
        num_rows: 28625
    })
    unlabelled_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
        num_rows: 0
    })
    unlabelled_test: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
        num_rows: 0
    })
    paper_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id', 'bool_label', 'wiki

In [47]:
for k,v in ds.items():
    print(set(v["label"]))

{'NOT ENOUGH INFO', 'REFUTES', 'SUPPORTS'}
{'NOT ENOUGH INFO', 'REFUTES', 'SUPPORTS'}
{''}
{''}
{'NOT ENOUGH INFO', 'REFUTES', 'SUPPORTS'}
{'NOT ENOUGH INFO', 'REFUTES', 'SUPPORTS'}


In [32]:
# ds_f = ds.filter(lambda x: "Ukrainian Soviet Socialist Republic" in x["claim"] )
ds_f = ds.filter(lambda x: x["label"] in ["SUPPORTS", "REFUTES"] )

Filter:   0%|          | 0/311431 [00:00<?, ? examples/s]

Filter:   0%|          | 0/37566 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19998 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19998 [00:00<?, ? examples/s]

Filter:   0%|          | 0/18999 [00:00<?, ? examples/s]

Filter:   0%|          | 0/18567 [00:00<?, ? examples/s]

In [33]:
ds_f

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
        num_rows: 263822
    })
    labelled_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
        num_rows: 28625
    })
    unlabelled_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
        num_rows: 0
    })
    unlabelled_test: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
        num_rows: 0
    })
    paper_dev: Dataset({
        features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
        num_rows: 14475
    })
    paper_test: Dataset({
        features: ['id', 'l

In [30]:
ds_f["labelled_dev"][0]

{'id': 163803,
 'label': 'SUPPORTS',
 'claim': 'Ukrainian Soviet Socialist Republic was a founding participant of the UN.',
 'evidence_annotation_id': 296950,
 'evidence_id': 288668,
 'evidence_wiki_url': 'Ukrainian_Soviet_Socialist_Republic',
 'evidence_sentence_id': 7}

In [25]:
for row in ds["train"]:
    if "Ukrainian Soviet Socialist Republic" in row["claim"]:
        print(row)

In [31]:
len(js["examples"])

6612

In [13]:
js["examples"][0]

{'id': 163803,
 'input': 'On June 2017, the following claim was made: Ukrainian Soviet Socialist Republic was a founding participant of the UN.\nQ: Was this claim true or false?',
 'target_scores': {'true': 1, 'false': 0}}

In [52]:
from datasets import Dataset
fever_orig = Dataset.from_json("shared_task_dev.jsonl")

Generating train split: 0 examples [00:00, ? examples/s]

Failed to load JSON from file '/Users/claudios/Projects/pdl/shared_task_dev.jsonl' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column(/evidence/[]/[]/[]) changed from number to string in row 0


DatasetGenerationError: An error occurred while generating the dataset

In [101]:
# df = pd.read_json("/Users/claudios/Downloads/wiki-pages/wiki-pages/wiki-003.jsonl", lines=True)
df = pd.read_json("shared_task_dev.jsonl", lines=True, encoding="utf-8")

In [125]:
import unicodedata

def remove_accents(x):
    return (unicodedata.normalize('NFD', x))
                       #.encode('ascii', 'ignore')
                       #.decode('utf-8'))
wiki_pages_df.index = wiki_pages_df.index.map(remove_accents)

In [102]:
def evidence_mapper(evidence):
    # print(len(evidence))
    evidences = set([ (x[2], x[3]) for x in evidence[0] if x[2] is not None])
    return list(evidences)
df["unique_evidence"] = df["evidence"].apply(evidence_mapper)

In [126]:
def evidence_mapper_sentence(evidences):
    lines = []
    for title,line in evidences:
        if title is None or line is None:
            continue
        title = remove_accents(title)

        if title not in wiki_pages_df.index:
            print(title)
            continue
        
        sentence = wiki_pages_df.loc[title]
        # if len(sentence) > 1:
            # print("wtf", sentence)
        if sentence["lines_split"] is not None and len(sentence["lines_split"]) > line:
            sentence = sentence["lines_split"][line]
            lines.append((title, line, sentence))
        else:
            print(sentence)
    return list(lines)
df["evidence_sentences"] = df["unique_evidence"].apply(evidence_mapper_sentence)

In [147]:
bigbench = pd.DataFrame.from_records(js["examples"])

In [151]:
bigbench.input.str.contains("Ukraine").any()

True

In [168]:
bigbench[bigbench.input.str.contains("There is a capital called Mogadishu")]

,id,input,target_scores


In [152]:
df["claim_in_bigbench"] = df["claim"].apply(lambda x: bigbench.input.str.contains(x).any())#any(x in y["input"] for y in js["examples"]))

In [169]:
df["evidence_sentence_count"] = df["evidence_sentences"].map(lambda x: len(x))

In [233]:
train_df = df[(~df.index.isin(bigbench.index)) & (df["evidence_sentence_count"] > 0)]
train_df

,verifiable,label,claim,evidence,unique_evidence,evidence_sentences,claim_in_bigbench,evidence_sentence_count
id,,,,,,,,
137334,VERIFIABLE,SUPPORTS,Fox 2000 Pictures released the film Soul Food.,"[[[289914, 283015, Soul_Food_-LRB-film-RRB-, 0...","[(Soul_Food_-LRB-film-RRB-, 0)]","[(Soul_Food_-LRB-film-RRB-, 0, Soul Food is a ...",False,1
111897,VERIFIABLE,REFUTES,Telemundo is a English-language television net...,"[[[131371, 146144, Telemundo, 0]], [[131371, 1...","[(Telemundo, 0)]","[(Telemundo, 0, Telemundo -LRB- -LSB- teleˈmun...",False,1
89891,VERIFIABLE,REFUTES,Damon Albarn's debut album was released in 2011.,"[[[107201, 120581, Damon_Albarn, 17]]]","[(Damon_Albarn, 17)]","[(Damon_Albarn, 17, He has also ventured into ...",False,1
181634,VERIFIABLE,SUPPORTS,There is a capital called Mogadishu.,"[[[210946, 218608, Mogadishu, 0]]]","[(Mogadishu, 0)]","[(Mogadishu, 0, Mogadishu -LRB- -LSB- ˌmɔːɡəˈd...",False,1
219028,VERIFIABLE,REFUTES,Savages was exclusively a German film.,"[[[260471, 258880, Savages_-LRB-2012_film-RRB-...","[(Savages_-LRB-2012_film-RRB-, 3)]","[(Savages_-LRB-2012_film-RRB-, 3, Savages is a...",False,1
...,...,...,...,...,...,...,...,...
27657,VERIFIABLE,SUPPORTS,Elizabeth Olsen is a person.,"[[[43790, 52421, Elizabeth_Olsen, 0]], [[43790...","[(Elizabeth_Olsen, 0)]","[(Elizabeth_Olsen, 0, Elizabeth Chase `` Lizzi...",False,1
91751,VERIFIABLE,SUPPORTS,Meghan Markle is an empath.,"[[[109126, 122822, Meghan_Markle, 5]]]","[(Meghan_Markle, 5)]","[(Meghan_Markle, 5, In 2016 , Markle was appoi...",False,1
108208,VERIFIABLE,SUPPORTS,The Times is a publication.,"[[[127022, 141521, The_Times, 0]], [[127022, 1...","[(The_Times, 0)]","[(The_Times, 0, The Times is a British daily -...",False,1


In [229]:
bigbench = bigbench.set_index("id")

In [230]:
bigbench

,input,target_scores
id,,
163803,"On June 2017, the following claim was made: Uk...","{'true': 1, 'false': 0}"
70041,"On June 2017, the following claim was made: 2 ...","{'true': 1, 'false': 0}"
202314,"On June 2017, the following claim was made: Th...","{'true': 0, 'false': 1}"
6032,"On June 2017, the following claim was made: Ar...","{'true': 0, 'false': 1}"
130048,"On June 2017, the following claim was made: Bu...","{'true': 0, 'false': 1}"
...,...,...
33339,"On June 2017, the following claim was made: Fi...","{'true': 0, 'false': 1}"
8538,"On June 2017, the following claim was made: He...","{'true': 0, 'false': 1}"
145641,"On June 2017, the following claim was made: Mi...","{'true': 0, 'false': 1}"


In [227]:
df = df.set_index("id")

In [231]:
df.loc[145641]

verifiable                                                        VERIFIABLE
label                                                                REFUTES
claim                                      Michael Hutchence died on a boat.
evidence                         [[[168967, 182663, Michael_Hutchence, 15]]]
unique_evidence                                    [(Michael_Hutchence, 15)]
evidence_sentences         [(Michael_Hutchence, 15, On the morning of 22 ...
claim_in_bigbench                                                       True
evidence_sentence_count                                                    1
Name: 145641, dtype: object

In [199]:
test_df

,label,claim,unique_evidence,evidence_sentences,evidence_sentence_count
id,,,,,
163803,True,Ukrainian Soviet Socialist Republic was a foun...,"[[Ukrainian_Soviet_Socialist_Republic, 7]]","[[Ukrainian_Soviet_Socialist_Republic, 7, The ...",1
70041,True,2 Hearts is a musical composition by Minogue.,"[[2_Hearts_-LRB-Kylie_Minogue_song-RRB-, 0]]","[[2_Hearts_-LRB-Kylie_Minogue_song-RRB-, 0, ``...",1
202314,False,The New Jersey Turnpike has zero shoulders.,"[[New_Jersey_Turnpike, 15]]","[[New_Jersey_Turnpike, 15, It was officially o...",1
6032,False,Aruba is the only ABC Island.,"[[ABC_islands_-LRB-Lesser_Antilles-RRB-, 0]]","[[ABC_islands_-LRB-Lesser_Antilles-RRB-, 0, Th...",1
130048,False,"Burbank, California has always been completely...","[[Burbank,_California, 7]]","[[Burbank,_California, 7, It was the location ...",1
...,...,...,...,...,...
33339,False,Firefox is the second most popular dog breed.,"[[Firefox, 13]]","[[Firefox, 13, , Firefox has between 9 % and 1...",1
8538,False,Hermit crabs are arachnids.,"[[Hermit_crab, 0], [Decapoda, 0]]","[[Hermit_crab, 0, Hermit crabs are decapod cru...",2
145641,False,Michael Hutchence died on a boat.,"[[Michael_Hutchence, 15]]","[[Michael_Hutchence, 15, On the morning of 22 ...",1


In [344]:
test_df = (df[(df.index.isin(bigbench.index)) & (df["evidence_sentence_count"] > 0)]
.drop(columns=["verifiable", "claim_in_bigbench", "evidence"]))
test_df["unique_evidence"] = test_df["unique_evidence"].map(lambda x: [ [str(title), str(sent_id)] for title,sent_id in x ])
test_df["evidence_sentences"] = test_df["evidence_sentences"].map(lambda x: [ [str(title), str(sent_id), str(sent)] for title,sent_id,sent in x ])
test_df["label"] = test_df["label"] == "SUPPORTS"
test_df.index = test_df.index.astype(pd.StringDtype())
test_df.claim = test_df.claim.astype(pd.StringDtype())
test_df["id"] = test_df.index
test_df.to_json("fever_test_df.json", orient='records', lines=True)

In [346]:
train_df = (
    df[(~df.index.isin(bigbench.index)) & (df["evidence_sentence_count"] > 0)]
    .drop(columns=["verifiable", "claim_in_bigbench", "evidence"])
)
train_df["unique_evidence"] = train_df["unique_evidence"].map(lambda x: [ [str(title), str(sent_id)] for title,sent_id in x ])
train_df["evidence_sentences"] = train_df["evidence_sentences"].map(lambda x: [ [str(title), str(sent_id), str(sent)] for title,sent_id,sent in x ])
train_df["label"] = train_df["label"] == "SUPPORTS"
train_df.index = train_df.index.astype(pd.StringDtype())
train_df.claim = train_df.claim.astype(pd.StringDtype())
train_df["id"] = train_df.index
train_df.to_json("fever_train_df.json", orient='records', lines=True)

In [237]:
train_df

,label,claim,unique_evidence,evidence_sentences,evidence_sentence_count
id,,,,,
137334,True,Fox 2000 Pictures released the film Soul Food.,"[[Soul_Food_-LRB-film-RRB-, 0]]","[[Soul_Food_-LRB-film-RRB-, 0, Soul Food is a ...",1
111897,False,Telemundo is a English-language television net...,"[[Telemundo, 0]]","[[Telemundo, 0, Telemundo -LRB- -LSB- teleˈmun...",1
89891,False,Damon Albarn's debut album was released in 2011.,"[[Damon_Albarn, 17]]","[[Damon_Albarn, 17, He has also ventured into ...",1
181634,True,There is a capital called Mogadishu.,"[[Mogadishu, 0]]","[[Mogadishu, 0, Mogadishu -LRB- -LSB- ˌmɔːɡəˈd...",1
219028,False,Savages was exclusively a German film.,"[[Savages_-LRB-2012_film-RRB-, 3]]","[[Savages_-LRB-2012_film-RRB-, 3, Savages is a...",1
...,...,...,...,...,...
27657,True,Elizabeth Olsen is a person.,"[[Elizabeth_Olsen, 0]]","[[Elizabeth_Olsen, 0, Elizabeth Chase `` Lizzi...",1
91751,True,Meghan Markle is an empath.,"[[Meghan_Markle, 5]]","[[Meghan_Markle, 5, In 2016 , Markle was appoi...",1
108208,True,The Times is a publication.,"[[The_Times, 0]]","[[The_Times, 0, The Times is a British daily -...",1


In [347]:
fever_train_ds = load_dataset("json", data_files={"train": "fever_train_df.json", "test": "fever_test_df.json"})
fever_train_ds.save_to_disk("var/fever_reprocessed")
fever_train_ds

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6720 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6612 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'claim', 'unique_evidence', 'evidence_sentences', 'evidence_sentence_count', 'id'],
        num_rows: 6720
    })
    test: Dataset({
        features: ['label', 'claim', 'unique_evidence', 'evidence_sentences', 'evidence_sentence_count', 'id'],
        num_rows: 6612
    })
})

In [238]:
fever_train_ds

DatasetDict({
    train: Dataset({
        features: ['label', 'claim', 'unique_evidence', 'evidence_sentences', 'evidence_sentence_count'],
        num_rows: 7179
    })
    test: Dataset({
        features: ['label', 'claim', 'unique_evidence', 'evidence_sentences', 'evidence_sentence_count'],
        num_rows: 6153
    })
})

In [213]:
fever_train_ds["train"]["evidence_sentences"]

[[['Giada_at_Home',
   '1',
   'It first aired on October 18 , 2008 on the Food Network .\tFood Network\tFood Network\n'],
  ['Food_Network',
   '0',
   'Food Network -LRB- legally known as Television Food Network -RRB- is an American basic cable and satellite television channel that is owned by Television Food Network , G.P. , a joint venture between Scripps Networks Interactive -LRB- which owns 70 % of the network -RRB- and the Tribune -LRB- FN -RRB- Cable Ventures Inc. -LRB- which owns the remaining 30 % -RRB- .\tScripps Networks Interactive\tScripps Networks Interactive\tAmerican\tUnited States\tbasic cable\tcable television\tsatellite television\tsatellite television\tchannel\tcable channel\tjoint venture\tjoint venture\n']],
 [['James_Taylor',
   '0',
   'James Vernon Taylor -LRB- born March 12 , 1948 -RRB- is an American singer-songwriter and guitarist .\tsinger-songwriter\tsinger-songwriter\tguitarist\tguitarist\n'],
  ['Hourglass_-LRB-James_Taylor_album-RRB-',
   '0',
   "Hour

In [174]:
train_df = train_df.astype(
    {"verifiable": }
)
train_df.dtypes

id                          int64
verifiable                 object
label                      object
claim                      object
evidence                   object
unique_evidence            object
evidence_sentences         object
claim_in_bigbench            bool
evidence_sentence_count     int64
dtype: object

In [158]:
df[(df.verifiable != "VERIFIABLE") & (df["claim_in_bigbench"] == True)]

,id,verifiable,label,claim,evidence,unique_evidence,evidence_sentences,claim_in_bigbench
10077,126261,NOT VERIFIABLE,NOT ENOUGH INFO,Pocahontas was the daughter of the president.,"[[[148109, None, None, None]]]","[(None, None)]",[],True
10083,201605,NOT VERIFIABLE,NOT ENOUGH INFO,A thunderstorm can also be called an hurricane.,"[[[237312, None, None, None]]]","[(None, None)]",[],True
10115,8317,NOT VERIFIABLE,NOT ENOUGH INFO,Donald Trump was a member of the Grateful Dead.,"[[[13518, None, None, None], [18925, None, Non...","[(None, None)]",[],True
10431,24075,NOT VERIFIABLE,NOT ENOUGH INFO,Musala is a valley.,"[[[40031, None, None, None]]]","[(None, None)]",[],True
10732,139985,NOT VERIFIABLE,NOT ENOUGH INFO,The Africa Cup of Nations had its schedule cha...,"[[[162951, None, None, None]]]","[(None, None)]",[],True
10933,201464,NOT VERIFIABLE,NOT ENOUGH INFO,A static web page is delivered to the user.,"[[[237129, None, None, None]]]","[(None, None)]",[],True
10986,94476,NOT VERIFIABLE,NOT ENOUGH INFO,Georgians arose from a place.,"[[[111911, None, None, None]]]","[(None, None)]",[],True
11227,164891,NOT VERIFIABLE,NOT ENOUGH INFO,Hezbollah is part of Iran.,"[[[189638, None, None, None]]]","[(None, None)]",[],True
11578,12819,NOT VERIFIABLE,NOT ENOUGH INFO,Internet access requires the use of computers.,"[[[28096, None, None, None]]]","[(None, None)]",[],True
11760,226844,NOT VERIFIABLE,NOT ENOUGH INFO,Sejong the Great was a queen.,"[[[270486, None, None, None]]]","[(None, None)]",[],True


In [106]:
wiki_pages_df.index = wiki_pages_df.index.str.encode('utf-8').str.decode('utf-8')

In [118]:
wiki_pages_df[wiki_pages_df.index.str.contains("Sim.*n_Bol.*var") ]

,text,lines,lines_split
id,,,
2009_Copa_Simón_Bolívar,The 2nd tier in the Bolivian Football pyramid ...,0\tThe 2nd tier in the Bolivian Football pyram...,[The 2nd tier in the Bolivian Football pyramid...
2010_Copa_Simón_Bolívar,This is the 22nd season of Copa Simon Bolivar ...,0\tThis is the 22nd season of Copa Simon Boliv...,[This is the 22nd season of Copa Simon Bolivar...
2008_Copa_Simón_Bolívar,"The tournament started in 1960 , initially onl...","0\tThe tournament started in 1960 , initially ...","[The tournament started in 1960 , initially on..."
Birthplace_of_Simón_Bolívar,The Birthplace of Simón Bolívar -LRB- Casa Nat...,0\tThe Birthplace of Simón Bolívar -LRB- Casa ...,[The Birthplace of Simón Bolívar -LRB- Casa Na...
Centro_Simón_Bolívar_Towers,The Centro Simón Bolívar Towers TCSB also know...,0\tThe Centro Simón Bolívar Towers TCSB also k...,[The Centro Simón Bolívar Towers TCSB also kno...
Colegio_La_Salle_Simón_Bolívar,Colegio La Salle Simón Bolívar is a private sc...,0\tColegio La Salle Simón Bolívar is a private...,[Colegio La Salle Simón Bolívar is a private s...
Colegio_Simón_Bolívar_-LRB-Simon_Bolivar_University-RRB-,Colegio Simón Bolívar -LRB- CSB -RRB- is a pri...,0\tColegio Simón Bolívar -LRB- CSB -RRB- is a ...,[Colegio Simón Bolívar -LRB- CSB -RRB- is a pr...
Colegio_Simón_Bolívar,Colegio Simón Bolívar may refer to : Colegio ...,0\tColegio Simón Bolívar may refer to :\tSimón...,[Colegio Simón Bolívar may refer to :\tSimón B...
Copa_Simón_Bolívar,The old 2nd tier in the Bolivian Football pyra...,0\tThe old 2nd tier in the Bolivian Football p...,[The old 2nd tier in the Bolivian Football pyr...


In [124]:
print("Simón_Bolívar".encode())
print("Simón_Bolívar".encode())
print(unicodedata.normalize('NFD', "Simón_Bolívar").encode())
print(unicodedata.normalize('NFD', "Simón_Bolívar").encode())

b'Sim\xc3\xb3n_Bol\xc3\xadvar'
b'Simo\xcc\x81n_Boli\xcc\x81var'
b'Simo\xcc\x81n_Boli\xcc\x81var'
b'Simo\xcc\x81n_Boli\xcc\x81var'


In [108]:
wiki_pages_df.loc["Simón_Bolívar".encode('utf-8').decode('utf-8')]

KeyError: 'Simón_Bolívar'

In [82]:
wiki_pages_df

,text,lines,lines_split
id,,,
b'',,,[]
b'1928_in_association_football',The following are the football -LRB- soccer -R...,0\tThe following are the football -LRB- soccer...,[The following are the football -LRB- soccer -...
b'1986_NBA_Finals',The 1986 NBA Finals was the championship round...,0\tThe 1986 NBA Finals was the championship ro...,[The 1986 NBA Finals was the championship roun...
b'1901_Villanova_Wildcats_football_team',The 1901 Villanova Wildcats football team repr...,0\tThe 1901 Villanova Wildcats football team r...,[The 1901 Villanova Wildcats football team rep...
b'1992_Northwestern_Wildcats_football_team',The 1992 Northwestern Wildcats team represente...,0\tThe 1992 Northwestern Wildcats team represe...,[The 1992 Northwestern Wildcats team represent...
...,...,...,...
b'Yuto_Agarie',Yuto Agarie -LRB- born 6 July 1993 -RRB- is a ...,0\tYuto Agarie -LRB- born 6 July 1993 -RRB- is...,[Yuto Agarie -LRB- born 6 July 1993 -RRB- is a...
b'Yume_1_Go',is the eleventh single by the Japanese rock ba...,0\tis the eleventh single by the Japanese rock...,[is the eleventh single by the Japanese rock b...
b'Yada_Yada_-LRB-album-RRB-',Yada Yada is the eighth studio album by Dutch ...,0\tYada Yada is the eighth studio album by Dut...,[Yada Yada is the eighth studio album by Dutch...


In [64]:
df["ev_len1"] = df["unique_evidence"].map(len)

In [65]:
df["ev_len1"].max()

20

In [66]:
df.iloc[19993]

id                                                              8538
verifiable                                                VERIFIABLE
label                                                        REFUTES
claim                                    Hermit crabs are arachnids.
evidence           [[[15450, 19262, Hermit_crab, 0], [15450, 1926...
ev_len                                                             4
unique_evidence                    [(Hermit_crab, 0), (Decapoda, 0)]
ev_len1                                                            2
Name: 19993, dtype: object

In [24]:
df.iloc[19993]['evidence']

[[[15450, 19262, 'Hermit_crab', 0], [15450, 19262, 'Decapoda', 0]],
 [[16110, 19977, 'Hermit_crab', 0]],
 [[17708, 21856, 'Hermit_crab', 0]],
 [[303930, 294824, 'Hermit_crab', 0],
  [303930, 294824, 'Decapoda', 0],
  [303930, 294824, 'Arachnid', 0]]]

In [97]:
wikipages = load_dataset("json", data_files="/Users/claudios/Downloads/wiki-pages/wiki-pages/wiki-*.jsonl", encoding="utf-8")

Resolving data files:   0%|          | 0/109 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [98]:
wikipages["train"] = wikipages["train"].map(lambda x: {
    "lines_split": list(filter(lambda x: x != '', re.split(r"\d+\t", x["lines"])))
    }, num_proc=32)

Map (num_proc=32):   0%|          | 0/5416537 [00:00<?, ? examples/s]

In [99]:
wiki_pages_df = wikipages["train"].to_pandas().set_index("id")

In [ ]:
hc = wikipages["train"].filter(lambda x: x["id"] == "Hermit_crab")

Filter:   0%|          | 0/5416537 [00:00<?, ? examples/s]

In [41]:
hc_lines = hc[0]["lines"]
hc_lines

'0\tHermit crabs are decapod crustaceans of the superfamily Paguroidea .\tdecapod\tDecapoda\n1\t\n2\t\n3\tMost of the approximately 1100 species possess an asymmetrical abdomen that is concealed in a scavenged gastropod shell carried around by the hermit crab .\tgastropod shell\tgastropod shell\n4\t'

In [43]:
import re
list(filter(lambda x: x != '', re.split(r"\d+\t", hc_lines)))

['Hermit crabs are decapod crustaceans of the superfamily Paguroidea .\tdecapod\tDecapoda\n',
 '\n',
 '\n',
 'Most of the approximately 1100 species possess an asymmetrical abdomen that is concealed in a scavenged gastropod shell carried around by the hermit crab .\tgastropod shell\tgastropod shell\n']

In [451]:
prompt = {"webact_simple3": "\nDetermine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. \nClaim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nAction 1: Search[Nikolaj Coster-Waldau]\nObservation 1: Nikolaj William Coster-Waldau (born 27 July 1970) is a Danish actor and producer. He graduated from the Danish National School of Performing Arts in Copenhagen in 1993,[1] and had his breakthrough role in Denmark with the film Nightwatch (1994). He played Jaime Lannister in the HBO fantasy drama series Game of Thrones, for which he received two Primetime Emmy Award nominations for Outstanding Supporting Actor in a Drama Series.. Coster-Waldau has appeared in numerous films in his native Denmark and Scandinavia, including Headhunters (2011) and A Thousand Times Good Night (2013). In the U.S, his debut film role was in the war film Black Hawk Down (2001), playing Medal of Honor recipient Gary Gordon.[2] He then played a detective in the short-lived Fox television series New Amsterdam (2008), and appeared in the 2009 Fox television film Virtuality, originally intended as a pilot.\nAction 2: Finish[SUPPORTS]\n\nClaim: Stranger Things is set in Bloomington, Indiana.\nAction 1: Search[Stranger Things]\nObservation 1: Stranger Things is an American science fiction horror drama television series created by the Duffer Brothers. Set in the 1980s, primarily in the fictional town of Hawkins, Indiana, the series centers on a number of mysteries and supernatural events occurring around the town and their impact on an ensemble of child and adult characters. \nAction 2: Finish[REFUTES]\n\nClaim: Beautiful reached number two on the Billboard Hot 100 in 2003.?\nAction 1: Search[Beautiful]\nObservation 1: Could not find [Beautiful]. Similar: ['Beautiful', 'Beautiful, Beautiful', 'A Beautiful Mind (film)', 'Beautiful (Christina Aguilera song)', 'Life Is Beautiful'].\nAction 2: Search[Beautiful (Christina Aguilera song)]\nObservation 2: \"Beautiful\" is a song recorded by American singer Christina Aguilera for her fourth studio album, Stripped (2002).\nAction 3: Lookup[Billboard Hot 100]\nObservation 3: (Result 1 / 3) The song peaked at number two on the Billboard Hot 100 in the United States, where it was certified Gold for 500,000 units shipped.\nAction 4: Finish[NOT ENOUGH INFO]\n\n", "cotqa_simple3": "Determine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. \nClaim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nThought: Nikolaj William Coster-Waldau appeared in the 2009 Fox television film Virtuality, so he has worked with the Fox Broadcasting Company.\nAnswer: SUPPORTS\n\nClaim: Stranger Things is set in Bloomington, Indiana.\nThought: Stranger Things is in the fictional town of Hawkins, Indiana, not in Bloomington, Indiana.\nAnswer:REFUTES\n\nClaim: Beautiful reached number two on the Billboard Hot 100 in 2003.?\nThought: The song peaked at number two on the Billboard Hot 100 in the United States, but not sure if it was in 2003.\nAnswer: NOT ENOUGH INFO\n", "webqa_simple3": "Determine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. \nClaim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nAnswer: SUPPORTS\n\nClaim: Stranger Things is set in Bloomington, Indiana.\nAnswer:REFUTES\n\nClaim: Beautiful reached number two on the Billboard Hot 100 in 2003.?\nAnswer: NOT ENOUGH INFO\n", "webthink_simple3": "\nDetermine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. \nClaim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nThought 1: I need to search Nikolaj Coster-Waldau and find if he has worked with the Fox Broadcasting Company.\nAction 1: Search[Nikolaj Coster-Waldau]\nObservation 1: Nikolaj William Coster-Waldau (born 27 July 1970) is a Danish actor and producer. He graduated from the Danish National School of Performing Arts in Copenhagen in 1993,[1] and had his breakthrough role in Denmark with the film Nightwatch (1994). He played Jaime Lannister in the HBO fantasy drama series Game of Thrones, for which he received two Primetime Emmy Award nominations for Outstanding Supporting Actor in a Drama Series.. Coster-Waldau has appeared in numerous films in his native Denmark and Scandinavia, including Headhunters (2011) and A Thousand Times Good Night (2013). In the U.S, his debut film role was in the war film Black Hawk Down (2001), playing Medal of Honor recipient Gary Gordon.[2] He then played a detective in the short-lived Fox television series New Amsterdam (2008), and appeared in the 2009 Fox television film Virtuality, originally intended as a pilot.\nThought 2: Because he \"appeared in the 2009 Fox television film Virtuality\", he should have worked with the Fox Broadcasting Company.\nAction 2: Finish[SUPPORTS]\n\nClaim: Stranger Things is set in Bloomington, Indiana.\nThought 1: I should search for Stranger Things, and see if it is set in Bloomington, Indiana.\nAction 1: Search[Stranger Things]\nObservation 1: Stranger Things is an American science fiction horror drama television series created by the Duffer Brothers. Set in the 1980s, primarily in the fictional town of Hawkins, Indiana, the series centers on a number of mysteries and supernatural events occurring around the town and their impact on an ensemble of child and adult characters. \nThought 2: The observation says that it is set in a \"fictional town of Hawkins, Indiana\", so it is not set in Bloomington.\nAction 2: Finish[REFUTES]\n\nClaim: Beautiful reached number two on the Billboard Hot 100 in 2003.?\nThought 1: I need to search the song Beautiful and find if it reached number two on the Billboard Hot 100 in 2003.\nAction 1: Search[Beautiful]\nObservation 1: Could not find [Beautiful]. Similar: ['Beautiful', 'Beautiful, Beautiful', 'A Beautiful Mind (film)', 'Beautiful (Christina Aguilera song)', 'Life Is Beautiful'].\nThought 2: From suggestions, I should search \"Beautiful (Christina Aguilera song)\" to find the song.\nAction 2: Search[Beautiful (Christina Aguilera song)]\nObservation 2: \"Beautiful\" is a song recorded by American singer Christina Aguilera for her fourth studio album, Stripped (2002).\nThought 3: It does not mention Billboard, so I need to look up \"Billboard Hot 100\" to find if it reached number two on it in 2003.\nAction 3: Lookup[Billboard Hot 100]\nObservation 3: (Result 1 / 3) The song peaked at number two on the Billboard Hot 100 in the United States, where it was certified Gold for 500,000 units shipped.\nThought 4: It only says the song peaked at number two on the Billboard Hot 100, but not if it was in 2003. I am not sure if this claim is true or not.\nAction 4: Finish[NOT ENOUGH INFO]\n\n"}
pprint(prompt)

{'cotqa_simple3': 'Determine if there is Observation that SUPPORTS or REFUTES '
                  'a Claim, or if there is NOT ENOUGH INFORMATION. \n'
                  'Claim: Nikolaj Coster-Waldau worked with the Fox '
                  'Broadcasting Company.\n'
                  'Thought: Nikolaj William Coster-Waldau appeared in the 2009 '
                  'Fox television film Virtuality, so he has worked with the '
                  'Fox Broadcasting Company.\n'
                  'Answer: SUPPORTS\n'
                  '\n'
                  'Claim: Stranger Things is set in Bloomington, Indiana.\n'
                  'Thought: Stranger Things is in the fictional town of '
                  'Hawkins, Indiana, not in Bloomington, Indiana.\n'
                  'Answer:REFUTES\n'
                  '\n'
                  'Claim: Beautiful reached number two on the Billboard Hot '
                  '100 in 2003.?\n'
                  'Thought: The song peaked at number two on the Billbo

In [1]:
react = {"webact_simple3": "\nDetermine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. \nClaim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nAction 1: Search[Nikolaj Coster-Waldau]\nObservation 1: Nikolaj William Coster-Waldau (born 27 July 1970) is a Danish actor and producer. He graduated from the Danish National School of Performing Arts in Copenhagen in 1993,[1] and had his breakthrough role in Denmark with the film Nightwatch (1994). He played Jaime Lannister in the HBO fantasy drama series Game of Thrones, for which he received two Primetime Emmy Award nominations for Outstanding Supporting Actor in a Drama Series.. Coster-Waldau has appeared in numerous films in his native Denmark and Scandinavia, including Headhunters (2011) and A Thousand Times Good Night (2013). In the U.S, his debut film role was in the war film Black Hawk Down (2001), playing Medal of Honor recipient Gary Gordon.[2] He then played a detective in the short-lived Fox television series New Amsterdam (2008), and appeared in the 2009 Fox television film Virtuality, originally intended as a pilot.\nAction 2: Finish[SUPPORTS]\n\nClaim: Stranger Things is set in Bloomington, Indiana.\nAction 1: Search[Stranger Things]\nObservation 1: Stranger Things is an American science fiction horror drama television series created by the Duffer Brothers. Set in the 1980s, primarily in the fictional town of Hawkins, Indiana, the series centers on a number of mysteries and supernatural events occurring around the town and their impact on an ensemble of child and adult characters. \nAction 2: Finish[REFUTES]\n\nClaim: Beautiful reached number two on the Billboard Hot 100 in 2003.?\nAction 1: Search[Beautiful]\nObservation 1: Could not find [Beautiful]. Similar: ['Beautiful', 'Beautiful, Beautiful', 'A Beautiful Mind (film)', 'Beautiful (Christina Aguilera song)', 'Life Is Beautiful'].\nAction 2: Search[Beautiful (Christina Aguilera song)]\nObservation 2: \"Beautiful\" is a song recorded by American singer Christina Aguilera for her fourth studio album, Stripped (2002).\nAction 3: Lookup[Billboard Hot 100]\nObservation 3: (Result 1 / 3) The song peaked at number two on the Billboard Hot 100 in the United States, where it was certified Gold for 500,000 units shipped.\nAction 4: Finish[NOT ENOUGH INFO]\n\n", "cotqa_simple3": "Determine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. \nClaim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nThought: Nikolaj William Coster-Waldau appeared in the 2009 Fox television film Virtuality, so he has worked with the Fox Broadcasting Company.\nAnswer: SUPPORTS\n\nClaim: Stranger Things is set in Bloomington, Indiana.\nThought: Stranger Things is in the fictional town of Hawkins, Indiana, not in Bloomington, Indiana.\nAnswer:REFUTES\n\nClaim: Beautiful reached number two on the Billboard Hot 100 in 2003.?\nThought: The song peaked at number two on the Billboard Hot 100 in the United States, but not sure if it was in 2003.\nAnswer: NOT ENOUGH INFO\n", "webqa_simple3": "Determine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. \nClaim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nAnswer: SUPPORTS\n\nClaim: Stranger Things is set in Bloomington, Indiana.\nAnswer:REFUTES\n\nClaim: Beautiful reached number two on the Billboard Hot 100 in 2003.?\nAnswer: NOT ENOUGH INFO\n", "webthink_simple3": "\nDetermine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. \nClaim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nThought 1: I need to search Nikolaj Coster-Waldau and find if he has worked with the Fox Broadcasting Company.\nAction 1: Search[Nikolaj Coster-Waldau]\nObservation 1: Nikolaj William Coster-Waldau (born 27 July 1970) is a Danish actor and producer. He graduated from the Danish National School of Performing Arts in Copenhagen in 1993,[1] and had his breakthrough role in Denmark with the film Nightwatch (1994). He played Jaime Lannister in the HBO fantasy drama series Game of Thrones, for which he received two Primetime Emmy Award nominations for Outstanding Supporting Actor in a Drama Series.. Coster-Waldau has appeared in numerous films in his native Denmark and Scandinavia, including Headhunters (2011) and A Thousand Times Good Night (2013). In the U.S, his debut film role was in the war film Black Hawk Down (2001), playing Medal of Honor recipient Gary Gordon.[2] He then played a detective in the short-lived Fox television series New Amsterdam (2008), and appeared in the 2009 Fox television film Virtuality, originally intended as a pilot.\nThought 2: Because he \"appeared in the 2009 Fox television film Virtuality\", he should have worked with the Fox Broadcasting Company.\nAction 2: Finish[SUPPORTS]\n\nClaim: Stranger Things is set in Bloomington, Indiana.\nThought 1: I should search for Stranger Things, and see if it is set in Bloomington, Indiana.\nAction 1: Search[Stranger Things]\nObservation 1: Stranger Things is an American science fiction horror drama television series created by the Duffer Brothers. Set in the 1980s, primarily in the fictional town of Hawkins, Indiana, the series centers on a number of mysteries and supernatural events occurring around the town and their impact on an ensemble of child and adult characters. \nThought 2: The observation says that it is set in a \"fictional town of Hawkins, Indiana\", so it is not set in Bloomington.\nAction 2: Finish[REFUTES]\n\nClaim: Beautiful reached number two on the Billboard Hot 100 in 2003.?\nThought 1: I need to search the song Beautiful and find if it reached number two on the Billboard Hot 100 in 2003.\nAction 1: Search[Beautiful]\nObservation 1: Could not find [Beautiful]. Similar: ['Beautiful', 'Beautiful, Beautiful', 'A Beautiful Mind (film)', 'Beautiful (Christina Aguilera song)', 'Life Is Beautiful'].\nThought 2: From suggestions, I should search \"Beautiful (Christina Aguilera song)\" to find the song.\nAction 2: Search[Beautiful (Christina Aguilera song)]\nObservation 2: \"Beautiful\" is a song recorded by American singer Christina Aguilera for her fourth studio album, Stripped (2002).\nThought 3: It does not mention Billboard, so I need to look up \"Billboard Hot 100\" to find if it reached number two on it in 2003.\nAction 3: Lookup[Billboard Hot 100]\nObservation 3: (Result 1 / 3) The song peaked at number two on the Billboard Hot 100 in the United States, where it was certified Gold for 500,000 units shipped.\nThought 4: It only says the song peaked at number two on the Billboard Hot 100, but not if it was in 2003. I am not sure if this claim is true or not.\nAction 4: Finish[NOT ENOUGH INFO]\n\n"}

In [3]:
pprint(react)

Pretty printing has been turned ON


In [4]:
react

{'webact_simple3': '\nDetermine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. \nClaim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nAction 1: Search[Nikolaj Coster-Waldau]\nObservation 1: Nikolaj William Coster-Waldau (born 27 July 1970) is a Danish actor and producer. He graduated from the Danish National School of Performing Arts in Copenhagen in 1993,[1] and had his breakthrough role in Denmark with the film Nightwatch (1994). He played Jaime Lannister in the HBO fantasy drama series Game of Thrones, for which he received two Primetime Emmy Award nominations for Outstanding Supporting Actor in a Drama Series.. Coster-Waldau has appeared in numerous films in his native Denmark and Scandinavia, including Headhunters (2011) and A Thousand Times Good Night (2013). In the U.S, his debut film role was in the war film Black Hawk Down (2001), playing Medal of Honor recipient Gary Gordon.[2] He then played a detective in 

In [5]:
react_naive = {"webthink_simple": "\nQuestion: Musician and satirist Allie Goertz wrote a song about the \"The Simpsons\" character Milhouse, who Matt Groening named after who?\nThought 1: I only need to search Milhouse and find who it is named after.\nAction 1: Search[Milhouse]\nObservation 1: Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening. Milhouse is Bart Simpson's best friend in Mrs. Krabappel's fourth grade class at Springfield Elementary School. He is an insecure, gullible, and less popular child than Bart who is often led into trouble by Bart, who takes advantage of his friend's na\u00c3\u00afvet\u00c3\u00a9. Milhouse is a regular target for school bully Nelson Muntz and his friends Jimbo Jones, Dolph Starbeam and Kearney Zzyzwicz. Milhouse has a crush on Bart's sister, Lisa, a common plot element.\nThought 2: The paragraph does not tell who Milhouse is named after, maybe I can look up \"named after\".\nAction 2: Lookup[named after]\nObservation 2: (Result 1 / 1) Milhouse was designed by Matt Groening for a planned series on NBC, which was abandoned.[4] The design was then used for a Butterfinger commercial, and it was decided to use the character in the series.[5][6] Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous. The name was the most \"unfortunate name Matt Groening could think of for a kid\".[1] Years earlier, in a 1986 Life in Hell comic entitled \"What to Name the Baby\", Groening listed Milhouse as a name \"no longer recommended\".[7] Milhouse is a favorite among the staff as Al Jean noted \"most of the writers are more like Milhouse than Bart\".[1] His last name was given to him by Howard Gewirtz, a freelance writer who wrote the episode \"Homer Defined\". Gewirtz got the name from one of his wife's friends.[8] According to Arden Myrin and Dana Gould (former writer and co-executive producer of The Simpsons), Rob Cohen (Simpsons writer) inspired Milhouse's look.[2]\nThought 3: Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.\nAction 3: Finish[Richard Nixon]\nQuestion: Guitars for Wounded Warriors is an album that was recorded in the village in which New York county?\nThought 1: I need to search Guitars for Wounded Warriors, then find in which New York county it was recorded.\nAction 1: Search[Guitars for Wounded Warriors]\nObservation 1: Guitars for Wounded Warriors is the sixth solo studio album by guitarist Brian Tarquin, released in August 2014 by Cleopatra Records. In 2016, it received a Best Compilation Album nomination by the Independent Music Awards. All tracks were recorded at Tarquin's Jungle Room Studios in New Paltz (village), New York. Being moved by the lack of support for returning veterans through his life Tarquin decided to compose a heavy instrumental rock album as a way to show his appreciation to all veterans. So once again he enlisted top notch players to guest with him on the album, Billy Sheehan on Black Hawk, Ron \"Bumblefoot\" Thal and Reb Beach on Taliban Terror, Steve Morse on Freedom, Chris Poland on Charlie Surfs and Hunting, Chuck Loeb on Escape Kabul, Hal Lindes on Sand & Blood, Gary Hoey on 5 Klicks To Hell and Baghdad, Alex De Rosso Dokken on Broken Arrow, and The Flyin' Ryan Brothers on Surgical Strike.  The entire album was engineered, produced and composed by Tarquin especially for each one of the guest guitarists. Partial proceeds are donated to the Fisher House Foundation from sales.\nThought 2: Guitars for Wounded Warriors was recorded in New Paltz (village), New York. I need to search New Paltz (village), New York and find which county it is in.\nAction 2: Search[New Paltz (village), New York]\nObservation 2: New Paltz is a village in Ulster County located in the U.S. state of New York. It is approximately 80 miles (130 km) north of New York City and 70 miles (110 km) south of Albany. The population was 7,324 at the 2020 census.[3]\nThought 3: New Paltz (village), New York is in Ulster County, so the answer is Ulster County.\nAction 3: Finish[Ulster County]\nQuestion: This American actor who starred in \"Veep\" was also in an action comedy film directed by Seth Rogen and Evan Goldberg heavily inspired by what?\nThought 1: There could be many American actors who starred in \"Veep\". So I can first search \"directed by Seth Rogen and Evan Goldberg\" and find action comedy films directed by Seth Rogen and Evan Goldberg.\nAction 1: Search[directed by Seth Rogen and Evan Goldberg]\nObservation 1: Could not find directed by Seth Rogen and Evan Goldberg. Similar: ['The Interview (redirect from The Interview (Seth Rogen film))', 'Evan Goldberg', 'Jay and Seth Versus the Apocalypse', 'This Is the End (category Films directed by Seth Rogen)', 'Seth Rogen filmography', 'Superbad (category Films with screenplays by Seth Rogen)', 'Pineapple Express (film) (category Films with screenplays by Seth Rogen)', 'Sausage Party (category Films produced by Seth Rogen)', 'Untitled Teenage Mutant Ninja Turtles film (category Films produced by Seth Rogen)', 'The Green Hornet (2011 film) (category Films produced by Seth Rogen)', 'Good Boys (film) (category Films produced by Seth Rogen)', 'Seth Rogen', 'Neighbors 2: Sorority Rising (category Films produced by Seth Rogen)', 'An American Pickle (category Films produced by Seth Rogen)', 'The Watch (2012 film) (category Films with screenplays by Seth Rogen)', 'Neighbors (2014 film) (category Films produced by Seth Rogen)', 'The Night Before (2015 film) (category Films produced by Seth Rogen)', 'Martin Starr', 'Preacher (TV series) (category Television shows directed by Steph Green)', 'Long Shot (2019 film) (category Films produced by Seth Rogen)'].\nThought 2: I can check search The Interview (Seth Rogen film), Jay and Seth Versus the Apocalypse, This Is the End (category Films directed by Seth Rogen) and find which is an action comedy film directed by Seth Rogen and Evan Goldberg.\nAction 2: Search[The Interview (Seth Rogen film)]\nObservation 2: The Interview is a 2014 action-adventure comedy film co-produced and directed by Seth Rogen and Evan Goldberg in their second directorial work, following This Is the End (2013). The screenplay was written by Dan Sterling, based on a story he co-wrote with Rogen and Goldberg. The film stars Rogen and James Franco as journalists who set up an interview with North Korean leader Kim Jong-un (Randall Park), and are then recruited by the CIA to assassinate him. The film is inspired by a 2012 Vice documentary.\nThought 3: The Interview is an action comedy film directed by Seth Rogen and Evan Goldberg, and it is inspired by a 2012 Vice documentary. So I can probably stop now without checking other films, and the answer is probably a 2012 Vice documentary.\nAction 3: Finish[a 2012 Vice documentary]\nQuestion: The head coach during the 1997\u201398 Indiana Pacers season retired as a player from what NBA team?\nThought 1: I can search Indiana Pacers, find the head coach during the 1997\u201398 season, then find from what NBA team it is retired as a player.\nAction 1: Search[Indiana Pacers]\nObservation 1: The Indiana Pacers are an American professional basketball team based in Indianapolis. The Pacers compete in the National Basketball Association (NBA) as a member of the league's Eastern Conference Central Division. The Pacers were first established in 1967 as a member of the American Basketball Association (ABA) and became a member of the NBA in 1976 as a result of the ABA\u00e2\u0080\u0093NBA merger. They play their home games at Gainbridge Fieldhouse. The team is named after the state of Indiana's history with the Indianapolis 500's pace cars and with the harness racing industry.[8][9]\nThought 2: I can look up \"1997\u201398\" to find the head coach during the 1997\u201398 season.\nAction 2: Lookup[1997\u201398]\nObservation 2: No more results.\nThought 3: I can look up \"1997\" instead to find the head coach during the 1997\u201398 season\nAction 3: Lookup[1997]\nObservation 3: (Result 1 / 5) In the 1997\u00e2\u0080\u009398 NBA season, Indiana native and former Boston Celtics great Larry Bird was hired as head coach. He led the Pacers to a 19-game improvement over the previous season, finishing 58\u00e2\u0080\u009324 \u00e2\u0080\u0093 at the time, the most the franchise had ever won as an NBA team, and tying the 1970\u00e2\u0080\u009371 ABA Pacers for the franchise record. Chris Mullin joined the team in the off-season and immediately became a valuable part of the Pacers lineup\u00e2\u0080\u0094and starting small forward. Assistant coaches Rick Carlisle, in charge of the offense, and Dick Harter, who coached the defense, were key in getting the most out of the Pacers' role players such as Dale Davis, Derrick McKey and a young Antonio Davis. Miller and Rik Smits both made the All-Star team that year, and in the playoffs, the Pacers breezed past the Cleveland Cavaliers and New York Knicks before falling to the Chicago Bulls in a seven-game Eastern Conference Finals.\nThought 4: In the 1997-98 season, former Boston Celtics great Larry Bird was hired as head coach. So the answer is Boston Celtics.\nAction 4: Finish[Boston Celtics]\nQuestion: What profession does Nicholas Ray and Elia Kazan have in common?\nThought 1: I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.\nAction 1: Search[Nicholas Ray]\nObservation 1: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 \u2013 June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause. He is appreciated for many narrative features produced between 1947 and 1963 including They Live By Night, In A Lonely Place, Johnny Guitar, and Bigger Than Life, as well as an experimental work produced throughout the 1970s titled We Can't Go Home Again, which was unfinished at the time of Ray's death.\nThought 2: Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions.\nAction 2: Search[Elia Kazan]\nObservation 2: Elia Kazan (/\u02c8i\u02d0li\u0259 k\u0259\u02c8z\u00e6n/;[1][2] born Elias Kazantzoglou (Greek: \u0397\u03bb\u03af\u03b1\u03c2 \u039a\u03b1\u03b6\u03b1\u03bd\u03c4\u03b6\u03cc\u03b3\u03bb\u03bf\u03c5);[3] September 7, 1909 \u2013 September 28, 2003) was an American film and theatre director, producer, screenwriter and actor, described by The New York Times as \"one of the most honored and influential directors in Broadway and Hollywood history\".[4]\nThought 3: Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.\nAction 3: Finish[director, screenwriter, actor]\nQuestion: Which magazine was started first Arthur's Magazine or First for Women?\nThought 1: I need to search Arthur's Magazine and First for Women, and find which was started first.\nAction 1: Search[Arthur's Magazine]\nObservation 1: Arthur's Magazine (1844\u00e2\u0080\u00931846) was an American literary periodical published in Philadelphia in the 19th century. Edited by Timothy Shay Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others.[1][2] In May 1846 it was merged into Godey's Lady's Book.[3]\nThought 2: Arthur's Magazine was started in 1844. I need to search First for Women next.\nAction 2: Search[First for Women]\nObservation 2: First for Women is a woman's magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989. It is based in Englewood Cliffs, New Jersey.[2] In 2011 the circulation of the magazine was 1,310,696 copies.[3]\nThought 3: First for Women was started in 1989. 1844 (Arthur's Magazine) < 1989 (First for Women), so Arthur's Magazine was started first.\nAction 3: Finish[Arthur's Magazine]\nQuestion: Were Pavel Urysohn and Leonid Levin known for the same type of work?\nThought 1: I need to search Pavel Urysohn and Leonid Levin, find their types of work, then find if they are the same.\nAction 1: Search[Pavel Urysohn]\nObservation 1: Pavel Samuilovich Urysohn (\u00d0\u009f\u00d0\u00b0\u00cc\u0081\u00d0\u00b2\u00d0\u00b5\u00d0\u00bb \u00d0\u00a1\u00d0\u00b0\u00d0\u00bc\u00d1\u0083\u00d0\u00b8\u00cc\u0081\u00d0\u00bb\u00d0\u00be\u00d0\u00b2\u00d0\u00b8\u00d1\u0087 \u00d0\u00a3\u00d1\u0080\u00d1\u008b\u00d1\u0081\u00d0\u00be\u00cc\u0081\u00d0\u00bd) (February 3, 1898 \u00e2\u0080\u0093 August 17, 1924) was a Soviet mathematician who is best known for his contributions in dimension theory, and for developing Urysohn's metrization theorem and Urysohn's lemma, both of which are fundamental results in topology. His name is also commemorated in the terms Urysohn universal space, Fr\u00c3\u00a9chet\u00e2\u0080\u0093Urysohn space, Menger\u00e2\u0080\u0093Urysohn dimension and Urysohn integral equation. He and Pavel Alexandrov formulated the modern definition of compactness in 1923.\nThought 2: Pavel Urysohn is a mathematician. I need to search Leonid Levin next and find its type of work.\nAction 2: Search[Leonid Levin]\nObservation 2: Leonid Anatolievich Levin (/le\u00c9\u00aa.o\u00ca\u008a\u00cb\u0088ni\u00cb\u0090d \u00cb\u0088l\u00c9\u009bv\u00c9\u00aan/ lay-oh-NEED LEV-in; Russian: \u00d0\u009b\u00d0\u00b5\u00d0\u00be\u00d0\u00bd\u00d0\u00b8\u00cc\u0081\u00d0\u00b4 \u00d0\u0090\u00d0\u00bd\u00d0\u00b0\u00d1\u0082\u00d0\u00be\u00cc\u0081\u00d0\u00bb\u00d1\u008c\u00d0\u00b5\u00d0\u00b2\u00d0\u00b8\u00d1\u0087 \u00d0\u009b\u00d0\u00b5\u00cc\u0081\u00d0\u00b2\u00d0\u00b8\u00d0\u00bd; Ukrainian: \u00d0\u009b\u00d0\u00b5\u00d0\u00be\u00d0\u00bd\u00d1\u0096\u00cc\u0081\u00d0\u00b4 \u00d0\u0090\u00d0\u00bd\u00d0\u00b0\u00d1\u0082\u00d0\u00be\u00cc\u0081\u00d0\u00bb\u00d1\u0096\u00d0\u00b9\u00d0\u00be\u00d0\u00b2\u00d0\u00b8\u00d1\u0087 \u00d0\u009b\u00d0\u00b5\u00cc\u0081\u00d0\u00b2\u00d1\u0096\u00d0\u00bd; born November 2, 1948) is a Soviet-American mathematician and computer scientist. \nThought 3: Leonid Levin is a mathematician and computer scientist. So Pavel Urysohn and Leonid Levin have the same type of work. \nAction 3: Finish[yes]\nQuestion: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?\nThought 1: I need to search Adam Clayton Powell and The Saimaa Gesture, and find which documentary is about Finnish rock groups.\nAction 1: Search[Adam Clayton Powell]\nObservation 1: Could not find [Adam Clayton Powell]. Similar: ['Adam Clayton Powell Jr.', 'Adam Clayton Powell Sr.', 'Adam Clayton Powell IV', 'Adam Clayton Powell III', 'Seventh Avenue (Manhattan) (redirect from Adam Clayton Powell Jr. Boulevard)', 'Adam Clayton Powell Jr. State Office Building', 'Isabel Washington Powell', 'Adam Powell', 'Adam Clayton Powell (film)', 'Giancarlo Esposito', 'Harlem', 'St. Nicholas Historic District', 'Abyssinian Baptist Church', 'Powell (surname)', 'Adam (musical)', 'Statue of Adam Clayton Powell Jr.', 'List of people from Harlem', 'Adam (given name)', 'Five-Percent Nation', 'List of American liberals'].\nThought 2: I can search Adam Clayton Powell (film) to find the documentary.\nAction 2: Search[Adam Clayton Powell (film)]\nObservation 2: Adam Clayton Powell is a 1989 American documentary film directed by Richard Kilberg.\nThe film is about the rise and fall of influential African-American politician Adam Clayton Powell Jr.[3][4] It was later aired as part of the PBS series The American Experience.\nThought 3: Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture.\nAction 3: Finish[The Saimaa Gesture]\n", "cotqa_simple": "\nQuestion: Musician and satirist Allie Goertz wrote a song about the \"The Simpsons\" character Milhouse, who Matt Groening named after who?\nThought: Let's think step by step. Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.\nAnswer: Richard Nixon\nQuestion: Guitars for Wounded Warriors is an album that was recorded in the village in which New York county?\nThought: Let's think step by step. Guitars for Wounded Warriors was recorded at Tarquin's Jungle Room Studios in New Paltz (village), New York. New Paltz is a village in Ulster County located in the U.S. state of New York. So the answer is Ulster County.\nAnswer: Ulster County\nQuestion: This American actor who starred in \"Veep\" was also in an action comedy film directed by Seth Rogen and Evan Goldberg heavily inspired by what?\nThought: Let's think step by step. The Interview is an action comedy film directed by Seth Rogen and Evan Goldberg. It is inspired by a 2012 Vice documentary. So the answer is a 2012 Vice documentary.\nAnswer: a 2012 Vice documentary\nQuestion: The head coach during the 1997\u201398 Indiana Pacers season retired as a player from what NBA team?\nThought: Let's think step by step. The head coach during the 1997\u201398 Indiana Pacers season was Larry Bird. Larry Bird retired as a player from Boston Celtics. So the answer is Boston Celtics.\nAnswer: Boston Celtics\nQuestion: What profession does Nicholas Ray and Elia Kazan have in common?\nThought: Let's think step by step. Professions of Nicholas Ray are director, screenwriter, and actor. Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.\nAnswer: director, screenwriter, actor\nQuestion: Which band was founded first, Hole, the rock band that Courtney Love was a frontwoman of, or The Wolfhounds?\nThought: Let's think step by step. Hole was founded in 1989. The Wolfhounds was founded in 1985. 1989 (Hole) > 1985 (The Wolfhounds), so The Wolfhounds was founded first.\nAnswer: The Wolfhounds\nQuestion: Were Pavel Urysohn and Leonid Levin known for the same type of work?\nThought: Let's think step by step. Pavel Urysohn is a mathematician. Leonid Levin is a mathematician and computer scientist. So Pavel Urysohn and Leonid Levin have the same type of work.\nAnswer: Yes\nQuestion: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?\nThought: Let's think step by step. Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture.\nAnswer: The Saimaa Gesture\n", "webqa_simple": "\nQuestion: Musician and satirist Allie Goertz wrote a song about the \"The Simpsons\" character Milhouse, who Matt Groening named after who?\nAnswer: Richard Nixon\nQuestion: Guitars for Wounded Warriors is an album that was recorded in the village in which New York county?\nAnswer: Ulster County\nQuestion: This American actor who starred in \"Veep\" was also in an action comedy film directed by Seth Rogen and Evan Goldberg heavily inspired by what?\nAnswer: 2012 Vice documentary\nQuestion: The head coach during the 1997\u201398 Indiana Pacers season retired as a player from what NBA team?\nAnswer: Boston Celtics\nQuestion: What profession does Nicholas Ray and Elia Kazan have in common?\nAnswer: director, screenwriter, actor\nQuestion: Which magazine was started first Arthur's Magazine or First for Women?\nAnswer: Arthur's Magazine\nQuestion: Were Pavel Urysohn and Leonid Levin known for the same type of work?\nAnswer: yes\nQuestion: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?\nAnswer: The Saimaa Gesture\n", "webthink_simple_3": "\nQuestion: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nThought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.\nAction 1: Search[Colorado orogeny]\nObservation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas. This took place from 1780 to 1650 million years ago (Mya), during the Paleoproterozoic (Statherian Period). It is recorded in the Colorado orogen, a >500-km-wide belt of oceanic arc rock that extends southward into New Mexico. The Colorado orogeny was likely part of the larger Yavapai orogeny.\nThought 2: It does not mention the eastern sector of the Colorado orogeny. I need to look up eastern sector.\nAction 2: Lookup[eastern sector]\nObservation 2: (Result 1 / 1) The Colorado orogen, formerly called the Colorado province, is a >500-km-wide belt of oceanic arc rock (1.78\u20131.65 Ga) that extends southward into New Mexico and composes a major part of the Proterozoic provinces of southwestern United States. This transcontinental collisional event occurred during the  Paleoproterozoic (Statherian Period).[1] The Wyoming sector of the Colorado orogeny was formerly called the Medicine Bow orogeny. The eastern sector extends into the High Plains and is called the Central Plains orogeny. The boundary between the Colorado orogeny and the Wyoming craton is the Cheyenne belt, a 5-km-wide mylonitic shear zone that verges northward. The Cheyenne belt transects and cuts off the south edge of the older Trans-Hudson orogeny.[2]\nThought 3: The eastern sector of Colorado orogeny extends into the High Plains, so High Plains is the area. I need to search High Plains and find its elevation range.\nAction 3: Search[High Plains]\nObservation 3: High Plains refers to one of two distinct land regions:\nThought 4: I need to search High Plains (United States).\nAction 4: Search[High Plains (United States)]\nObservation 4: The High Plains are a subregion of the Great Plains, mainly in the Western United States, but also partly in the Midwest states of Nebraska, Kansas, and South Dakota, generally encompassing the western part of the Great Plains before the region reaches the Rocky Mountains. The High Plains are located in eastern Montana, southeastern Wyoming, southwestern South Dakota, western Nebraska, eastern Colorado, western Kansas, eastern New Mexico, western Oklahoma, and to just south of the Texas Panhandle.[4] The southern region of the Western High Plains ecology region contains the geological formation known as Llano Estacado which can be seen from a short distance or on satellite maps.[5] From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]\nThought 5: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.\nAction 5: Finish[1,800 to 7,000 ft]\nQuestion: Guitars for Wounded Warriors is an album that was recorded in the village in which New York county?\nThought 1: I need to search Guitars for Wounded Warriors, then find in which New York county it was recorded.\nAction 1: Search[Guitars for Wounded Warriors]\nObservation 1: Guitars for Wounded Warriors is the sixth solo studio album by guitarist Brian Tarquin, released in August 2014 by Cleopatra Records. In 2016, it received a Best Compilation Album nomination by the Independent Music Awards. All tracks were recorded at Tarquin's Jungle Room Studios in New Paltz (village), New York. Being moved by the lack of support for returning veterans through his life Tarquin decided to compose a heavy instrumental rock album as a way to show his appreciation to all veterans. So once again he enlisted top notch players to guest with him on the album, Billy Sheehan on Black Hawk, Ron \"Bumblefoot\" Thal and Reb Beach on Taliban Terror, Steve Morse on Freedom, Chris Poland on Charlie Surfs and Hunting, Chuck Loeb on Escape Kabul, Hal Lindes on Sand & Blood, Gary Hoey on 5 Klicks To Hell and Baghdad, Alex De Rosso Dokken on Broken Arrow, and The Flyin' Ryan Brothers on Surgical Strike.  The entire album was engineered, produced and composed by Tarquin especially for each one of the guest guitarists. Partial proceeds are donated to the Fisher House Foundation from sales.\nThought 2: Guitars for Wounded Warriors was recorded in New Paltz (village), New York. I need to search New Paltz (village), New York and find which county it is in.\nAction 2: Search[New Paltz (village), New York]\nObservation 2: New Paltz is a village in Ulster County located in the U.S. state of New York. It is approximately 80 miles (130 km) north of New York City and 70 miles (110 km) south of Albany. The population was 7,324 at the 2020 census.[3]\nThought 3: New Paltz (village), New York is in Ulster County, so the answer is Ulster County.\nAction 3: Finish[Ulster County]\nQuestion: Which magazine was started first Arthur's Magazine or First for Women?\nThought 1: I need to search Arthur's Magazine and First for Women, and find which was started first.\nAction 1: Search[Arthur's Magazine]\nObservation 1: Arthur's Magazine (1844\u00e2\u0080\u00931846) was an American literary periodical published in Philadelphia in the 19th century. Edited by Timothy Shay Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others.[1][2] In May 1846 it was merged into Godey's Lady's Book.[3]\nThought 2: Arthur's Magazine was started in 1844. I need to search First for Women next.\nAction 2: Search[First for Women]\nObservation 2: First for Women is a woman's magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989. It is based in Englewood Cliffs, New Jersey.[2] In 2011 the circulation of the magazine was 1,310,696 copies.[3]\nThought 3: First for Women was started in 1989. 1844 (Arthur's Magazine) < 1989 (First for Women), so Arthur's Magazine was started first.\nAction 3: Finish[Arthur's Magazine]\n", "webthink_simple6": "\nQuestion: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nThought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.\nAction 1: Search[Colorado orogeny]\nObservation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.\nThought 2: It does not mention the eastern sector. So I need to look up eastern sector.\nAction 2: Lookup[eastern sector]\nObservation 2: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.\nThought 3: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.\nAction 3: Search[High Plains]\nObservation 3: High Plains refers to one of two distinct land regions:\nThought 4: I need to instead search High Plains (United States).\nAction 4: Search[High Plains (United States)]\nObservation 4: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]\nThought 5: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.\nAction 5: Finish[1,800 to 7,000 ft]\nQuestion: Musician and satirist Allie Goertz wrote a song about the \"The Simpsons\" character Milhouse, who Matt Groening named after who?\nThought 1: The question simplifies to \"The Simpsons\" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.\nAction 1: Search[Milhouse]\nObservation 1: Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.\nThought 2: The paragraph does not tell who Milhouse is named after, maybe I can look up \"named after\".\nAction 2: Lookup[named after]\nObservation 2: (Result 1 / 1) Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous. \nThought 3: Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.\nAction 3: Finish[Richard Nixon]\nQuestion: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?\nThought 1: I need to search Adam Clayton Powell and The Saimaa Gesture, and find which documentary is about Finnish rock groups.\nAction 1: Search[Adam Clayton Powell]\nObservation 1: Could not find [Adam Clayton Powell]. Similar: ['Adam Clayton Powell III', 'Seventh Avenue (Manhattan)', 'Adam Clayton Powell Jr. State Office Building', 'Isabel Washington Powell', 'Adam Powell', 'Adam Clayton Powell (film)', 'Giancarlo Esposito'].\nThought 2: To find the documentary, I can search Adam Clayton Powell (film).\nAction 2: Search[Adam Clayton Powell (film)]\nObservation 2: Adam Clayton Powell is a 1989 American documentary film directed by Richard Kilberg.\nThe film is about the rise and fall of influential African-American politician Adam Clayton Powell Jr.[3][4] It was later aired as part of the PBS series The American Experience.\nThought 3: Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture.\nAction 3: Finish[The Saimaa Gesture]\nQuestion: What profession does Nicholas Ray and Elia Kazan have in common?\nThought 1: I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.\nAction 1: Search[Nicholas Ray]\nObservation 1: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 \u2013 June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.\nThought 2: Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions.\nAction 2: Search[Elia Kazan]\nObservation 2: Elia Kazan was an American film and theatre director, producer, screenwriter and actor.\nThought 3: Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.\nAction 3: Finish[director, screenwriter, actor]\nQuestion: Which magazine was started first Arthur's Magazine or First for Women?\nThought 1: I need to search Arthur's Magazine and First for Women, and find which was started first.\nAction 1: Search[Arthur's Magazine]\nObservation 1: Arthur's Magazine (1844-\u0080\u00931846) was an American literary periodical published in Philadelphia in the 19th century. \nThought 2: Arthur's Magazine was started in 1844. I need to search First for Women next.\nAction 2: Search[First for Women]\nObservation 2: First for Women is a woman's magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989. \nThought 3: First for Women was started in 1989. 1844 (Arthur's Magazine) < 1989 (First for Women), so Arthur's Magazine was started first.\nAction 3: Finish[Arthur's Magazine]\nQuestion: Were Pavel Urysohn and Leonid Levin known for the same type of work?\nThought 1: I need to search Pavel Urysohn and Leonid Levin, find their types of work, then find if they are the same.\nAction 1: Search[Pavel Urysohn]\nObservation 1: Pavel Samuilovich Urysohn (February 3, 1898 \u00e2\u0080\u0093 August 17, 1924) was a Soviet mathematician who is best known for his contributions in dimension theory.\nThought 2: Pavel Urysohn is a mathematician. I need to search Leonid Levin next and find its type of work.\nAction 2: Search[Leonid Levin]\nObservation 2: Leonid Anatolievich Levin is a Soviet-American mathematician and computer scientist. \nThought 3: Leonid Levin is a mathematician and computer scientist. So Pavel Urysohn and Leonid Levin have the same type of work. \nAction 3: Finish[yes]\n", "webact_simple6": "Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nAction 1: Search[Colorado orogeny]\nObservation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.\nAction 2: Lookup[eastern sector]\nObservation 2: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.\nAction 3: Search[High Plains]\nObservation 3: High Plains refers to one of two distinct land regions:\nAction 4: Search[High Plains (United States)]\nObservation 4: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]\nAction 5: Finish[1,800 to 7,000 ft]\nQuestion: Musician and satirist Allie Goertz wrote a song about the \"The Simpsons\" character Milhouse, who Matt Groening named after who?\nAction 1: Search[Milhouse]\nObservation 1: Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.\nAction 2: Lookup[named after]\nObservation 2: (Result 1 / 1) Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous. \nAction 3: Finish[Richard Nixon]\nQuestion: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?\nAction 1: Search[Adam Clayton Powell]\nObservation 1: Could not find [Adam Clayton Powell]. Similar: ['Adam Clayton Powell III', 'Seventh Avenue (Manhattan)', 'Adam Clayton Powell Jr. State Office Building', 'Isabel Washington Powell', 'Adam Powell', 'Adam Clayton Powell (film)', 'Giancarlo Esposito'].\nAction 2: Search[Adam Clayton Powell (film)]\nObservation 2: Adam Clayton Powell is a 1989 American documentary film directed by Richard Kilberg.\nThe film is about the rise and fall of influential African-American politician Adam Clayton Powell Jr.[3][4] It was later aired as part of the PBS series The American Experience.\nAction 3: Finish[The Saimaa Gesture]\nQuestion: What profession does Nicholas Ray and Elia Kazan have in common?\nAction 1: Search[Nicholas Ray]\nObservation 1: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 \u2013 June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.\nAction 2: Search[Elia Kazan]\nObservation 2: Elia Kazan was an American film and theatre director, producer, screenwriter and actor.\nAction 3: Finish[director, screenwriter, actor]\nQuestion: Which magazine was started first Arthur's Magazine or First for Women?\nAction 1: Search[Arthur's Magazine]\nObservation 1: Arthur's Magazine (1844-\u0080\u00931846) was an American literary periodical published in Philadelphia in the 19th century. \nAction 2: Search[First for Women]\nObservation 2: First for Women is a woman's magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989. \nAction 3: Finish[Arthur's Magazine]\nQuestion: Were Pavel Urysohn and Leonid Levin known for the same type of work?\nAction 1: Search[Pavel Urysohn]\nObservation 1: Pavel Samuilovich Urysohn (February 3, 1898 \u00e2\u0080\u0093 August 17, 1924) was a Soviet mathematician who is best known for his contributions in dimension theory.\nAction 2: Search[Leonid Levin]\nObservation 2: Leonid Anatolievich Levin is a Soviet-American mathematician and computer scientist. \nAction 3: Finish[yes]\n\n", "cotqa_simple6": "Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nThought: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.\nAnswer: 1,800 to 7,000 ft\nQuestion: Musician and satirist Allie Goertz wrote a song about the \"The Simpsons\" character Milhouse, who Matt Groening named after who?\nThought: Let's think step by step. Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.\nAnswer: Richard Nixon\nQuestion: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?\nThought: Let's think step by step. Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture.\nAnswer: The Saimaa Gesture\nQuestion: What profession does Nicholas Ray and Elia Kazan have in common?\nThought: Let's think step by step. Professions of Nicholas Ray are director, screenwriter, and actor. Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.\nAnswer: director, screenwriter, actor\nQuestion: Which magazine was started first Arthur's Magazine or First for Women?\nThought: Let's think step by step. Arthur's Magazine was started in 1844. First for Women was started in 1989. 1844 (Arthur's Magazine) < 1989 (First for Women), so Arthur's Magazine was started first.\nAnswer: Arthur's Magazine\nQuestion: Were Pavel Urysohn and Leonid Levin known for the same type of work?\nThought: Let's think step by step. Pavel Urysohn is a mathematician. Leonid Levin is a mathematician and computer scientist. So Pavel Urysohn and Leonid Levin have the same type of work.\nAnswer: Yes\n", "webqa_simple6": "Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nAnswer: 1,800 to 7,000 ft\nQuestion: Musician and satirist Allie Goertz wrote a song about the \"The Simpsons\" character Milhouse, who Matt Groening named after who?\nAnswer: Richard Nixon\nQuestion: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?\nAnswer: The Saimaa Gesture\nQuestion: What profession does Nicholas Ray and Elia Kazan have in common?\nAnswer: director, screenwriter, actor\nQuestion: Which magazine was started first Arthur's Magazine or First for Women?\nAnswer: Arthur's Magazine\nQuestion: Were Pavel Urysohn and Leonid Levin known for the same type of work?\nAnswer: Yes\n"}

In [7]:
react_naive

{'webthink_simple': '\nQuestion: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?\nThought 1: I only need to search Milhouse and find who it is named after.\nAction 1: Search[Milhouse]\nObservation 1: Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening. Milhouse is Bart Simpson\'s best friend in Mrs. Krabappel\'s fourth grade class at Springfield Elementary School. He is an insecure, gullible, and less popular child than Bart who is often led into trouble by Bart, who takes advantage of his friend\'s naÃ¯vetÃ©. Milhouse is a regular target for school bully Nelson Muntz and his friends Jimbo Jones, Dolph Starbeam and Kearney Zzyzwicz. Milhouse has a crush on Bart\'s sister, Lisa, a common plot element.\nThought 2: The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

In [19]:
wikipages.save_to_disk("var/wiki-pages")

Saving the dataset (0/15 shards):   0%|          | 0/5416537 [00:00<?, ? examples/s]

In [20]:
wikipages

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'lines'],
        num_rows: 5416537
    })
})